In [1]:
import pandas as pd
import gmaps
from lest_converter import lest_geo

# INPUTS
GOOGLE_API_KEY = 'AI...'  # Google API key
VIEW_TYPE = 'county'  # Pick from ['city', 'country']; 
                      # 'city' is static and for close-up views, 'county' (default) is dissipating 

gmaps.configure(api_key=GOOGLE_API_KEY) 

def lest_to_centered_geo(xs: str, ys: str) -> (float, float):
    """
    Covert L-Est coordinate ranges xs, ys to WGS84 centered coordinate (x, y).
    :param xs: str, L-Est coordinate range (X), e.g. '6534000-6534499'  
    :param ys: str, L-Est coordinate range (Y), e.g. '479000-479499'
    :return: (float, float), s tuple of WGS84 coordinates, e.g. (58.94683558380662, 23.639500327741143)
    """
    if type(xs) is str and type(ys) is str and '-' in xs and '-' in ys:
        get_center_coordinate = lambda c: sum(map(int, c.strip().split('-')))/2
        return lest_geo(get_center_coordinate(xs), get_center_coordinate(ys))
    return (None, None)

In [2]:
df_vara = pd.read_csv('vara_1.csv', header=0, sep='\t', usecols=['Lest_X', 'Lest_Y'])
df_avalik = pd.read_csv('avalik_1.csv', header=0, sep='\t', usecols=['Lest_X', 'Lest_Y'])

df = pd.concat([df_vara, df_avalik])

# Convert L-Est coordinate ranges (df[['Lest_X', 'Lest_Y']]) to centered WGS84 coordinate (df[['X', 'Y']]).
df['X'] = df.apply(lambda row: lest_to_centered_geo(row['Lest_X'], row['Lest_Y'])[0], axis=1)
df['Y'] = df.apply(lambda row: lest_to_centered_geo(row['Lest_X'], row['Lest_Y'])[1], axis=1)
df = df.dropna(subset=['X', 'Y'])  # Drop rows where 'X'|'Y' is NaN; gmaps doesn't support NaN-coordinates.
df.head()

,Lest_X,Lest_Y,X,Y
0,6589000-6589499,686500-686999,59.399968,27.288689
1,6584500-6584999,539000-539499,59.398862,24.690903
2,6589500-6589999,737500-737999,59.378958,28.185276
3,6589000-6589499,543500-543999,59.438813,24.771020
4,6473000-6473999,527000-527999,58.401052,24.470347


In [3]:
fig = gmaps.figure()
locations = df[['X', 'Y']]
example_params = {
    'city': {'point_radius': 0.0075, 'max_intensity': 150, 'dissipating': False},
    'county': {'point_radius': 29, 'max_intensity': 150, 'dissipating': True}
}
heatmap_layer = gmaps.heatmap_layer(locations, **example_params[VIEW_TYPE])
fig.add_layer(heatmap_layer)
fig

Figure()